In [57]:
import numpy as np
from scipy.integrate import quad, dblquad
from sympy import diff
from scipy.special import gamma

The cluster redshift, the angular size diameter, the kpc/arcmin to cm conversions, and the solar mass will be used often, so they are defined here. We also define the critical density at the cluster redshift.

In [42]:
z = 0.546    # redshift of MACS J0717.5+3745
kpc = 3.086e21    # 1 kpc in cm
d_A = 1.3e6 * kpc    # cm
arcmin_to_cm = 383.3 * kpc    # 1 arcmin = 383.3 kpc
M_sun = 2e33    # solar mass in grams
rho_crit = 1.7e-29    # g cm**-3

The temperature of the group is between 3.4 and 4.5 keV (1-sigma confidence ranges).

In [31]:
T_low = 3.4    # keV
T_high = 4.5    # keV

The region from which the spectrum of the group was extracted is an ellipse with major axis 0.47 arcmin and minor axis 0.34 arcmin.

In [32]:
a = 0.47 * arcmin_to_cm    # cm
b = 0.34 * arcmin_to_cm    # cm

We consider two possible 3D geometries for the region chosen: a prolate spheroid with the major axis in the plane of the sky and aligned with the filament, and an oblate spheroid with the minor axis in the plane of the sky and perpendicular to the filament. The volumes of these two regions are:

In [33]:
V1 = 4./3. * np.pi * a * b * a    # cm**3
V2 = 4./3. * np.pi * a * b * b    # cm**3
print("The volumes of the spheroids considered are V1 = %.3f Mpc**3 and V2 = %.3f Mpc**3." 
          % (V1/kpc**3/1e9, V2/kpc**3/1e9))

The volumes of the spheroids considered are V1 = 0.018 Mpc**3 and V2 = 0.013 Mpc**3.


The normalization of the group's spectrum is:

In [34]:
norm = 1.8e-4    # cm**-5 arcmin**-2

Assuming the density is constant in our region of interest, the Xspec normalization is defined as 

norm = n_e \* n_H \* cyl_V / ( 1e14 \* area \* pi \* D_A\*\*2 \* (1+z)\*\*2 )

We assume the hydrogen number density relates to the electron number density as: n_e = 1.2 n_H. For simplicity, we'll also assume a volume equal to the average of the two volumes calculated above. Therefore, the hydrogen number density is:

In [41]:
m_H = 1.7e-24    # grams
area = np.pi * a * b / arcmin_to_cm**2
V = (V1 + V2) / 2.
n_H = np.sqrt( norm * 1e14 * area * 4 * np.pi * d_A**2 * (1+z)**2 / (1.2 * V) )
rho_H = n_H * m_H
delta = rho_H / rho_crit

print("The mean mass density in the region of the group is %.2g g/cm**3 \
and it corresponds to an overdensity of ~%.0f." % (rho_H, delta))

The mean mass density in the region of the group is 4.8e-27 g/cm**3 and it corresponds to an overdensity of ~285.


The density corresponds to a gas mass of:

In [46]:
M_gas = rho_H * V / M_sun
print("The gas mass of the group is ~%.1g Msun." % M_gas)

The gas mass of the group is ~1e+12 Msun.


Based on a beta model fitted to the group's surface brightness profile, the central surface brightness is 7.3e-15 erg/s/arcmin\*\*2. This yields a central density:

In [66]:
S0 = 7.3e-15    # erg/s/arcmin**2
S0_cgs = 1.6e-5 #S0 / arcmin_to_cm**2
rc = 0.04 * arcmin_to_cm    # core radius in cm
beta = 0.4

cooling_function_low = 1/1.2 * (8.6e-3 * T_low**-1.7 + 5.8e-2 * T_low**0.5 + 6.4e-2) * 1e-22
cooling_function_high = 1/1.2 * (8.6e-3 * T_high**-1.7 + 5.8e-2 * T_high**0.5 + 6.4e-2) * 1e-22
n_high = np.sqrt(S0_cgs/ np.sqrt(np.pi) / rc / cooling_function_low / gamma(3.*beta - 0.5) * gamma(3.*beta))
n_low = np.sqrt(S0_cgs/ np.sqrt(np.pi) / rc / cooling_function_high / gamma(3.*beta - 0.5) * gamma(3.*beta))
print(n_low, n_high)

0.00293727297009 0.00306824557367


Under the assumption of hydrostatic equilibrium, the total mass of the group is defined by the equation is section 5.5.5 of Sarazin (1998).

In [ ]:
def mass(r, T):